In [3]:
import sys
print(sys.executable)

/usr/local/bin/python3


In [4]:
import torch
print(torch.__version__)  # coCheck PyTorch version
print(torch.cuda.is_available())  # Should return True




2.6.0
False


In [20]:
import requests
from bs4 import BeautifulSoup

def extract_text_from_url(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Check for HTTP errors
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Extract visible text from paragraphs and headings
        text = " ".join([p.get_text() for p in soup.find_all(["p", "h1", "h2", "h3", "h4", "h5", "h6"])])
        return text
    except Exception as e:
        print(f"Failed to fetch {url}: {e}")
        return None

# Example: Scrape multiple URLs
urls = [
    "https://en.wikipedia.org/wiki/Hampi",
    "https://whc.unesco.org/en/list/241/#",
    "https://openthemagazine.com/essay/the-untold-history-of-hampi/",
    #"https://www.opindia.com/2018/02/the-ruins-of-hampi-a-song-of-hope-and-a-story-of-treachery-then-and-now/",
    "https://en.wikipedia.org/wiki/Virupaksha_Temple,_Hampi",
    "https://www.thrillophilia.com/questions/why-is-hampi-called-the-city-of-ruins",
    "https://unacademy.com/content/bank-exam/study-material/general-awareness/group-of-monuments-at-hampi/#:~:text=Hampi%2C%20also%20known%20as%20the,like%20the%20lotus%20and%20corbels.",
    "https://vajiramandravi.com/upsc-daily-current-affairs/prelims-pointers/key-facts-about-hampi/",
    "https://www.britannica.com/place/Hampi",
    "https://www.karnataka.com/hampi/about-hampi/#google_vignette"
]
documents = {url: extract_text_from_url(url) for url in urls}

# Remove empty results
documents = {url: text for url, text in documents.items() if text}



In [21]:
from sentence_transformers import SentenceTransformer
import numpy as np
import torch

# Load a pre-trained embedding model and move it to GPU
model = SentenceTransformer("all-MiniLM-L6-v2", device="cuda" if torch.cuda.is_available() else "cpu")
print("Model loaded successfully!")

# Example embeddings (ensure `documents` is defined in your code)
embeddings = np.array([model.encode(text, device=model.device) for text in documents.values()])
print("Embeddings shape:", embeddings.shape)

Model loaded successfully!
Embeddings shape: (9, 384)


In [22]:
import chromadb

# Initialize ChromaDB client
chroma_client = chromadb.PersistentClient(path="./chroma_db")

# Create a collection
collection = chroma_client.get_or_create_collection(name="text_embeddings")

# Add data to the database
for i, (url, text) in enumerate(documents.items()):
    collection.add(
        ids=[str(i)],  
        documents=[text], 
        embeddings=[embeddings[i].tolist()]
    )




print("Stored in ChromaDB!")


Insert of existing embedding ID: 0
Add of existing embedding ID: 0
Insert of existing embedding ID: 1
Add of existing embedding ID: 1
Insert of existing embedding ID: 2
Add of existing embedding ID: 2
Insert of existing embedding ID: 3
Add of existing embedding ID: 3
Insert of existing embedding ID: 4
Add of existing embedding ID: 4
Insert of existing embedding ID: 5
Add of existing embedding ID: 5
Insert of existing embedding ID: 6
Add of existing embedding ID: 6


Stored in ChromaDB!


In [23]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
import uuid  # For generating unique IDs

# Use Hugging Face embeddings instead of OpenAI
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Load stored vectors
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=embedding_model)
query_store = Chroma(persist_directory="./hampi_queries_db", embedding_function=embedding_model)
# Use Groq API key
groq_api_key = "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"

# Define Retrieval-Augmented Generation (RAG) pipeline using Groq
qa = RetrievalQA.from_chain_type(
    llm=ChatGroq(model="llama3-8b-8192", groq_api_key=groq_api_key),  # Change model as needed
    retriever=vectorstore.as_retriever()
)

def is_relevant_to_hampi(prompt):
    keywords = ["Hampi", "monument", "Vijayanagara", "temple"]
    return any(keyword.lower() in prompt.lower() for keyword in keywords)

# # Query the database
prompt = input("Enter the prompt: ")

if is_relevant_to_hampi(prompt):
    result = qa.run(prompt)
    print(result)
else:
    print("Your query is not relevant to Hampi cultural sites.")

Hampi is a village located in the eastern part of Karnataka, India. It was the capital of the Vijayanagara Empire, one of the most powerful and wealthy empires in Indian history. Hampi is a UNESCO World Heritage Site and is considered one of the most important archaeological sites in the world.

Hampi was the capital of the Vijayanagara Empire from 1336 to 1565, and during that time it was a major center of trade, commerce, and culture. The city was known for its stunning architecture, which was a blend of Dravidian and Islamic styles. The city was also famous for its temples, which were built to honor the Hindu gods and goddesses.

The city was founded by the founder of the Vijayanagara Empire, Harihara, and his brother Bukka in the 14th century. During the reign of the Vijayanagara Empire, Hampi was the center of power and culture, and it attracted scholars, artists, and traders from all over the world.

However, in 1565, the city was attacked and destroyed by the Deccan Sultanates, 